In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import helper_mate as h
import helper as hd
from colorscheme import *

from gt_learner import GT_learner
import gr_em_learner as gr_em

In [ ]:
# Data parameters
BLOCK_SIZE = 1
N_BATCHES = 8
ALPHA_LIST = [0,90]
N_RUNS = 10

# Agent parameters
SIGMA_R = 3.0
PP_THRESHOLD = 0.4
D = 10
EM_SIZE = 1
#N_PARTICLES = 256

In [ ]:
# Generate N_RUNS datasets
datasets = [h.generate_batch_data(ALPHA_LIST, BLOCK_SIZE, N_BATCHES) for i in range(N_RUNS)]

# Define models to be tested
model_set = ['x', 'y', '1x2D', '2x1D_bg']

# Compute mllhs
results = {'gt': [], 'gr': [], 'em': []}
pbar = tf.keras.utils.Progbar(N_RUNS)
for data in datasets:
    result = {}
    # Ground truth learner
    result['gt'] = GT_learner(data, SIGMA_R, model_set)
    results['gt'].append(result['gt'])

    # Generative replay learner
    result['gr'] = gr_em.GR_EM_learner(data, SIGMA_R, model_set, verbose = False,
                        EM_size_limit = 0, pp_thr = PP_THRESHOLD, D = D, task_angles_in_data = ALPHA_LIST)
    results['gr'].append(result['gr'])

    # Episodic learner
    result['em'] = gr_em.GR_EM_learner(data, SIGMA_R, model_set, verbose = False,
                    EM_size_limit = EM_SIZE, pp_thr = PP_THRESHOLD, D = D, task_angles_in_data = ALPHA_LIST)
    results['em'].append(result['em'])

    pbar.add(1)

In [ ]:
# Compute switching times
switch_times = {}
for agent in ['gt', 'gr', 'em']:
    switch_times[agent] = [hd.model_change_time(results[agent][i], model_set[-1])
                                 for i in range(N_RUNS)]

In [ ]:
# Scatter plot of switching times with some noise added to points to avoid overlapping
T = BLOCK_SIZE * (N_BATCHES + 1)
epsilon = 0.3
plt.figure(figsize=(6,6))
plt.scatter(switch_times['gt'] + epsilon*np.random.randn(N_RUNS), switch_times['gr'] + epsilon*np.random.randn(N_RUNS))
plt.scatter(switch_times['gt'] + epsilon*np.random.randn(N_RUNS), switch_times['em'] + epsilon*np.random.randn(N_RUNS), color = 'r')
plt.plot([0, T], [0, T], '--')
plt.xlabel('GT switching time')
plt.ylabel('GR/EM switching time')
plt.xlim([0, T])
plt.ylim([0, T])
#add legend
plt.legend(['GR vs GT', 'EM vs GT'])
plt.show()

In [ ]:
# Scatter plot of switching times
T = BLOCK_SIZE * (N_BATCHES + 1)
plt.figure(figsize=(6,6))
plt.scatter(switch_times['gt'], switch_times['gr'])
plt.scatter(switch_times['gt'], switch_times['em'], color = 'r')
plt.plot([0, T], [0, T], '--')
plt.xlabel('GT switching time')
plt.ylabel('GR/EM switching time')
plt.xlim([0, T])
plt.ylim([0, T])

# Join corresponding points wit lines
for i in range(N_RUNS):
    plt.plot([switch_times['gt'][i], switch_times['gt'][i]],
             [switch_times['gr'][i], switch_times['em'][i]], color = 'gray', linewidth = 0.5)
             
plt.show()

In [ ]:
# GR vs EM switching time
T = BLOCK_SIZE * (N_BATCHES + 1)
plt.figure(figsize=(6,6))
plt.scatter(switch_times['gr'], switch_times['em'])
plt.plot([0, T], [0, T], '--')
plt.xlabel('GR switching time')
plt.ylabel('EM switching time')
plt.xlim([0, T])
plt.ylim([0, T])
plt.show()

In [ ]:
i = 5
learning_dicts = [results['gt'][i], results['gr'][i], results['em'][i]]
hd.plot_mllh_curves_subpanels(learning_dicts, model_set, T=len(data['c']), color_dict=model_colors_gergo, figsize=(15,4), data=datasets[0], markersize=5)

In [ ]:
# Plot mllhs
hd.plot_mmllh_curves(results['gt'][i], model_set, T=len(data['c']), color_dict=model_colors_gergo, figsize=(15,4), data=datasets[0])
hd.plot_mmllh_curves(results['gr'][i], model_set, T=len(data['c']), color_dict=model_colors_gergo, figsize=(15,4), data=datasets[0])
hd.plot_mmllh_curves(results['em'][i], model_set, T=len(data['c']), color_dict=model_colors_gergo, figsize=(15,4), data=datasets[0])

In [ ]:
switch_times['gt']